<a href="https://colab.research.google.com/github/unicamp-dl/IA025_2022S1/blob/main/ex08/Fernando_Fortes_Granado/Exerc%C3%ADcio_8_Fernando_Fortes_Granado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# import sys
# sys.path.append('/content/drive/My Drive/IA025/aula_8')

In [3]:
nome = "Fernando Fortes Granado"
print(f'Meu nome é {nome}')

Meu nome é Fernando Fortes Granado


#  Exercício: Modelo de Linguagem com auto-atenção

Este exercício é similar ao da Aula 7, mas iremos agora treinar uma rede neural *com auto-atenção* para prever a próxima palavra de um texto, data as palavras anteriores como entrada. 

Na camada de auto-atenção, não se esqueça de implementar:
- Embeddings de posição
- Projeções lineares (WQ, WK, WV, WO)
- Conexões residuais
- Camada de feed forward (2-layer MLP)



O dataset usado neste exercício (BrWaC) possui um tamanho razoável e você vai precisar rodar seus experimentos com GPU.

Alguns conselhos úteis:
- **ATENÇÃO:** o dataset é bem grande. Não dê comando de imprimí-lo.
- Durante a depuração, faça seu dataset ficar bem pequeno, para que a depuração seja mais rápida e não precise de GPU. Somente ligue a GPU quando o seu laço de treinamento já está funcionando
- Não deixe para fazer esse exercício na véspera. Ele é trabalhoso.

In [4]:
# iremos utilizar a biblioteca dos transformers para ter acesso ao tokenizador do BERT.
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 8.2 MB/s 
     |████████████████████████████████| 596 kB 49.3 MB/s 
     |████████████████████████████████| 6.6 MB 47.5 MB/s 
     |████████████████████████████████| 86 kB 7.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Importação dos pacotes

In [5]:
import collections
import itertools
import functools
import math
import random

import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook


In [6]:
# Check which GPU we are using
!nvidia-smi

Thu May 26 00:21:32 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


## Implementação do MyDataset

In [8]:
from typing import List


def tokenize(text: str, tokenizer):
    return tokenizer(text, return_tensors=None, add_special_tokens=False).input_ids


class MyDataset():
    def __init__(self, texts: List[str], tokenizer, context_size: int):
        # Escreva seu código aqui
        self.x = []
        self.y = []
        for text in texts:
          tokens = tokenize(text, tokenizer)
          for i in range(max(len(tokens) - context_size, 0)):
            self.x.append(tokens[i:i + context_size])
            self.y.append(tokens[i + context_size])
        self.x = torch.tensor(self.x, dtype=torch.long)
        self.y = torch.tensor(self.y, dtype=torch.long)

    def __len__(self):
        # Escreva seu código aqui
        return len(self.x)

    def __getitem__(self, idx):
        # Escreva seu código aqui
        return self.x[idx], self.y[idx]

## Testando se a implementação do MyDataset está correta

In [9]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

dummy_texts = ['Eu gosto de correr', 'Ela gosta muito de comer pizza']

dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=tokenizer, context_size=3)
dummy_loader = DataLoader(dummy_dataset, batch_size=6, shuffle=False)
assert len(dummy_dataset) == 5
print('passou no assert de tamanho do dataset')

first_batch_input, first_batch_target = next(iter(dummy_loader))

correct_first_batch_input = torch.LongTensor(
    [[ 3396, 10303,   125],
     [ 1660,  5971,   785],
     [ 5971,   785,   125],
     [  785,   125,  1847],
     [  125,  1847, 13779]])

correct_first_batch_target = torch.LongTensor([13239,   125,  1847, 13779, 15616])

assert torch.equal(first_batch_input, correct_first_batch_input)
print('Passou no assert de input')
assert torch.equal(first_batch_target, correct_first_batch_target)
print('Passou no assert de target')

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

passou no assert de tamanho do dataset
Passou no assert de input
Passou no assert de target


# Carregamento do dataset 

Iremos usar uma pequena amostra do dataset [BrWaC](https://www.inf.ufrgs.br/pln/wiki/index.php?title=BrWaC) para treinar e avaliar nosso modelo de linguagem.

In [10]:
!wget -nc https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula7/sample_brwac.txt

--2022-05-26 00:21:38--  https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula7/sample_brwac.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.197.128, 74.125.195.128, 142.250.99.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.197.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 123983611 (118M) [text/plain]
Saving to: ‘sample_brwac.txt’

sample_brwac.txt    100%[===================>] 118.24M   201MB/s    in 0.6s    

2022-05-26 00:21:39 (201 MB/s) - ‘sample_brwac.txt’ saved [123983611/123983611]



In [11]:
# Load datasets
context_size = 9

valid_examples = 1000
test_examples = 1000
texts = open('sample_brwac.txt').readlines()

# print('Truncating for debugging purposes.')
# texts = texts[:500]  

training_texts = texts[:-(valid_examples + test_examples)]
valid_texts = texts[-(valid_examples + test_examples):-test_examples]
test_texts = texts[-test_examples:]

training_dataset = MyDataset(texts=training_texts, tokenizer=tokenizer, context_size=context_size)
valid_dataset = MyDataset(texts=valid_texts, tokenizer=tokenizer, context_size=context_size)
test_dataset = MyDataset(texts=test_texts, tokenizer=tokenizer, context_size=context_size)

In [12]:
print(f'training examples: {len(training_dataset)}')
print(f'valid examples: {len(valid_dataset)}')
print(f'test examples: {len(test_dataset)}')

training examples: 25682513
valid examples: 1128869
test examples: 1113359


In [13]:
class LanguageModel(torch.nn.Module):

    def __init__(self, vocab_size, context_size, embedding_dim):
        """
        Implements the Self-attention, decoder-only."

        Args:
            vocab_size (int): Size of the input vocabulary.
            context_size (int): Size of the sequence to consider as context for prediction.
            embedding_dim (int): Dimension of the embedding layer for each word in the context.
        """
        # Escreva seu código aqui.
        super().__init__()
        self.token_embeddings = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)
        self.positional_embeddings = nn.Embedding(num_embeddings=context_size, embedding_dim=embedding_dim)
        self.attention_head = AttentionHead(embedding_dim)
        self.linear_1 = nn.Linear(embedding_dim, embedding_dim)
        self.linear_2 = nn.Linear(embedding_dim, embedding_dim)
        self.linear_3 = nn.Linear(embedding_dim, vocab_size)

    def forward(self, inputs):
        """
        Args:
            inputs is a LongTensor of shape (batch_size, context_size)
            
        Returns:
            logits of shape (batch_size, vocab_size)
        """
        # Escreva seu código aqui.
        token_embeddings = self.token_embeddings(inputs)
        embeddings = token_embeddings + torch.unsqueeze(self.positional_embeddings.weight, 0)  # shape (batch, context_size, embedding_dim)
        E = self.attention_head(embeddings)  # shape (batch, embedding_dim)
        output = self.linear_3(nn.functional.relu(self.linear_2(nn.functional.relu(self.linear_1(E))) + E))
        return output

class AttentionHead(torch.nn.Module):
    def __init__(self, embedding_dim):
        super().__init__()
        self.proj_keys = nn.Linear(embedding_dim, embedding_dim)  # Escolhi deixar bias=True pois nao ha batch norm
        self.proj_values = nn.Linear(embedding_dim, embedding_dim)
        self.proj_queries = nn.Linear(embedding_dim, embedding_dim)
        self.W0 = nn.Linear(embedding_dim, embedding_dim)

    def forward(self, inputs):
        # shape (batch, context_size, embedding_dim)
        batch_size, context_size, embedding_dim = inputs.shape
        last_tokens_embedding = inputs[:, -1, :]
        input_embeddings = inputs.view(-1, embedding_dim)

        keys = self.proj_keys(input_embeddings).view(inputs.shape)  # shape (batch, context_size, embedding_dim)
        values = self.proj_values(input_embeddings).view(inputs.shape)  # shape (batch, context_size, embedding_dim)
        queries = self.proj_queries(last_tokens_embedding).view(batch_size, 1, embedding_dim)  # shape (batch, 1, embedding_dim)

        attention_logits = torch.matmul(queries, torch.transpose(keys, 1, 2)) / math.sqrt(embedding_dim)  # shape (batch, 1, context_size)
        attention_probs = nn.functional.softmax(attention_logits, dim=-1).view(batch_size, context_size, 1)  # shape (batch, context_size, 1)

        E = torch.sum(attention_probs * values, axis=1)  # shape (batch, embedding_dim)
        return nn.functional.relu(self.W0(E))

## Teste o modelo com um exemplo

In [14]:
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=64,
).to(device)

sample_train, _ = next(iter(DataLoader(training_dataset)))
sample_train_gpu = sample_train.to(device)
model(sample_train_gpu).shape

torch.Size([1, 29794])

In [15]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of model parameters: {num_params}')

Number of model parameters: 3868962


## Assert da Perplexidade


In [16]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)


def perplexity(logits, target):
    """
    Computes the perplexity.

    Args:
        logits: a FloatTensor of shape (batch_size, vocab_size)
        target: a LongTensor of shape (batch_size,)

    Returns:
        A float corresponding to the perplexity
    """
    loss = nn.functional.cross_entropy(logits, target, reduction='mean')
    return torch.exp(loss)


n_examples = 1000

sample_train, target_token_ids = next(iter(DataLoader(training_dataset, batch_size=n_examples)))
sample_train_gpu = sample_train.to(device)
target_token_ids = target_token_ids.to(device)
logits = model(sample_train_gpu)

my_perplexity = perplexity(logits=logits, target=target_token_ids)

print(f'my perplexity:              {int(my_perplexity)}')
print(f'correct initial perplexity: {tokenizer.vocab_size}')

assert math.isclose(my_perplexity, tokenizer.vocab_size, abs_tol=7000)
print('Passou o no assert da perplexidade')

my perplexity:              30158
correct initial perplexity: 29794
Passou o no assert da perplexidade


## Laço de Treinamento e Validação

In [22]:
def checkpoint(model, optimizer, step, n_examples, loss, path):
    # Additional information
    STEP = step
    PATH = path
    LOSS = loss
    N_EXAMPLES = n_examples

    torch.save({
                'step': STEP,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': LOSS,
                'n_examples': N_EXAMPLES,
                }, PATH)
    
def resume_checkpoint(model, optimizer, path):
    try:
        checkpoint = torch.load(path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        step = checkpoint['step']
        loss = checkpoint['loss']
        n_examples = checkpoint['n_examples']
        print("Loaded model.")
    except:
        print("Could not load model.")
        step = 0
        loss = 9999
        n_examples = 0
    return step, n_examples, loss

max_examples = 1_000_000_000
eval_every_steps = 10000
lr = 3e-4


model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=128,
).to(device)

train_loader = DataLoader(training_dataset, batch_size=4096, shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=4096)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)


def train_step(input, target):
    model.train()
    model.zero_grad()

    logits = model(input.to(device))
    loss = nn.functional.cross_entropy(logits, target.to(device))
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input, target):
    model.eval()
    logits = model(input)
    loss = nn.functional.cross_entropy(logits, target)
    return loss.item()


train_losses = []
n_examples = 0
step = 0
path = 'drive/MyDrive/IA025/aula_8/model_checkpoint.pt'
step, n_examples, loss = resume_checkpoint(model, optimizer, path)
while n_examples < max_examples:
    for input, target in train_loader:
        loss = train_step(input.to(device), target.to(device)) 
        train_losses.append(loss)
        
        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(input.to(device), target.to(device))
                    for input, target in validation_loader]))

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []
            checkpoint(model, optimizer, step, n_examples, loss, path)

        n_examples += len(input)  # Increment of batch size
        step += 1
        if n_examples >= max_examples:
            break



Loaded model.
120000 steps; 399360000 examples so far; train ppl: 163.51, valid ppl: 187.76
130000 steps; 440320000 examples so far; train ppl: 161.01, valid ppl: 184.75
140000 steps; 481280000 examples so far; train ppl: 158.35, valid ppl: 182.39
150000 steps; 522240000 examples so far; train ppl: 155.74, valid ppl: 179.70


KeyboardInterrupt: ignored

## Avaliação final no dataset de teste


Bonus: o modelo com menor perplexidade no dataset de testes ganhará 0.5 ponto na nota final.

In [23]:
test_loader = DataLoader(test_dataset, batch_size=64)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(input.to(device), target.to(device))
        for input, target in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

test perplexity: 181.53830572254483


## Teste seu modelo com uma sentença

Escolha uma sentença gerada pelo modelo que ache interessante.

In [24]:
prompt = 'Eu gosto de comer pizza pois me faz'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-context_size:]  # Usamos apenas os últimos <context_size> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Eu gosto de comer pizza pois me faz com
Eu gosto de comer pizza pois me faz com a
Eu gosto de comer pizza pois me faz com a sua
Eu gosto de comer pizza pois me faz com a sua vida
Eu gosto de comer pizza pois me faz com a sua vida.
Eu gosto de comer pizza pois me faz com a sua vida. O
Eu gosto de comer pizza pois me faz com a sua vida. O que
Eu gosto de comer pizza pois me faz com a sua vida. O que é
Eu gosto de comer pizza pois me faz com a sua vida. O que é o
Eu gosto de comer pizza pois me faz com a sua vida. O que é o que
Eu gosto de comer pizza pois me faz com a sua vida. O que é o que é
Eu gosto de comer pizza pois me faz com a sua vida. O que é o que é o
Eu gosto de comer pizza pois me faz com a sua vida. O que é o que é o que
Eu gosto de comer pizza pois me faz com a sua vida. O que é o que é o que é
Eu gosto de comer pizza pois me faz com a sua vida. O que é o que é o que é o
Eu gosto de comer pizza pois me faz com a sua vida. O que é o que é o que é o que
Eu gosto de comer piz

In [27]:
prompt = 'Todos os dias eu acordo cedo pois Deus ajuda quem'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-context_size:]  # Usamos apenas os últimos <context_size> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Todos os dias eu acordo cedo pois Deus ajuda quem não
Todos os dias eu acordo cedo pois Deus ajuda quem não tem
Todos os dias eu acordo cedo pois Deus ajuda quem não tem a
Todos os dias eu acordo cedo pois Deus ajuda quem não tem a ver
Todos os dias eu acordo cedo pois Deus ajuda quem não tem a ver com
Todos os dias eu acordo cedo pois Deus ajuda quem não tem a ver com a
Todos os dias eu acordo cedo pois Deus ajuda quem não tem a ver com a presença
Todos os dias eu acordo cedo pois Deus ajuda quem não tem a ver com a presença de
Todos os dias eu acordo cedo pois Deus ajuda quem não tem a ver com a presença de um
Todos os dias eu acordo cedo pois Deus ajuda quem não tem a ver com a presença de um novo
Todos os dias eu acordo cedo pois Deus ajuda quem não tem a ver com a presença de um novo sistema
Todos os dias eu acordo cedo pois Deus ajuda quem não tem a ver com a presença de um novo sistema de
Todos os dias eu acordo cedo pois Deus ajuda quem não tem a ver com a presença de um novo s

In [26]:
prompt = 'Você gosta de futebol? Eu assisto todos os jogos do'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-context_size:]  # Usamos apenas os últimos <context_size> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Você gosta de futebol? Eu assisto todos os jogos do que
Você gosta de futebol? Eu assisto todos os jogos do que se
Você gosta de futebol? Eu assisto todos os jogos do que se refere
Você gosta de futebol? Eu assisto todos os jogos do que se refere à
Você gosta de futebol? Eu assisto todos os jogos do que se refere à saúde
Você gosta de futebol? Eu assisto todos os jogos do que se refere à saúde,
Você gosta de futebol? Eu assisto todos os jogos do que se refere à saúde, a
Você gosta de futebol? Eu assisto todos os jogos do que se refere à saúde, a partir
Você gosta de futebol? Eu assisto todos os jogos do que se refere à saúde, a partir de
Você gosta de futebol? Eu assisto todos os jogos do que se refere à saúde, a partir de um
Você gosta de futebol? Eu assisto todos os jogos do que se refere à saúde, a partir de um dos
Você gosta de futebol? Eu assisto todos os jogos do que se refere à saúde, a partir de um dos mais
Você gosta de futebol? Eu assisto todos os jogos do que se refere à saú